In [18]:
import numpy as np
import lda
import lda.datasets
from nltk.stem import SnowballStemmer
import nltk

In [19]:
import json
with open('ac2-combined-ideas.json', 'r') as f:
    ac1_dict = json.load(f)["@graph"]
    content_array = list(map(lambda idea: idea["content"], ac1_dict));

with open('patent_25k.csv', 'r') as f:
    title = []
    content = []
    for line in f:
        arr = line.split("\t")
        title += [arr[7]]
        content += [arr[6]]
        
print(len(content),len(title))
content_array += content
#with open('ac2-abstract-ideas.json', 'r') as f:
#    ac2_dict = json.load(f)["@graph"]
#content_array += list(map(lambda idea: idea["content"], ac2_dict));
#print(content_array)


25000 25000


In [22]:
englishStemmer=SnowballStemmer("english",  ignore_stopwords=True)
content_array = [" ".join([englishStemmer.stem(x) for x in nltk.word_tokenize(c)]) for c in content_array]

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = CountVectorizer(stop_words="english", max_df=0.1)
print("nr_of_sprowords:",len(vectorizer.get_stop_words()))
X = vectorizer.fit_transform(content_array)
vocab = vectorizer.get_feature_names()
print("nr_ideas:",len(content_array),"vocab:",len(vocab), X.shape)

nr_of_sprowords: 318
nr_ideas: 25419 vocab: 10530 (25419, 10530)


In [24]:
new_model = lda.LDA(n_topics=10, n_iter=1500, random_state=1)
new_model.fit(X)  # model.fit_transform(X) is also available
topic_word = new_model.topic_word_  # model.components_ also works
n_top_words = 8
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

INFO:lda:n_documents: 25419
INFO:lda:vocab_size: 10530
INFO:lda:n_words: 139493
INFO:lda:n_topics: 10
INFO:lda:n_iter: 1500
/usr/local/lib/python3.7/site-packages/lda/utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):
INFO:lda:<0> log likelihood: -1614323
INFO:lda:<10> log likelihood: -1208382
INFO:lda:<20> log likelihood: -1147285
INFO:lda:<30> log likelihood: -1121639
INFO:lda:<40> log likelihood: -1109925
INFO:lda:<50> log likelihood: -1103976
INFO:lda:<60> log likelihood: -1098398
INFO:lda:<70> log likelihood: -1095122
INFO:lda:<80> log likelihood: -1091792
INFO:lda:<90> log likelihood: -1089013
INFO:lda:<100> log likelihood: -1086425
INFO:lda:<110> log likelihood: -1085786
INFO:lda:<120> log likelihood: -1083949
INFO:lda:<130> log likelihood: -1082653
INFO:lda:<140> log likelihood: -10828

Topic 0: display semiconductor manufactur light electron structur imag having
Topic 1: composit process materi use produc product contain manufactur
Topic 2: use composit thereof treatment compound cell treat acid
Topic 3: measur optic sensor use detect high magnet imag
Topic 4: imag process communic control data inform wireless medium
Topic 5: power vehicl control electr circuit drive electron oper
Topic 6: data network manag base content comput servic user
Topic 7: assembl tool having structur support mechan machin member
Topic 8: use detect monitor object technolog track movement vehicl
Topic 9: control engin heat air gas valv fluid fuel


In [25]:
doc_topic = new_model.doc_topic_
for i in range(10):
    print(np.sort(doc_topic[i])[::-1][:3])
    arr = np.arange(len(doc_topic))[np.argsort(doc_topic[i])][::-1][:3]
    print("{} (top topic: {})".format(content_array[i], arr))

[0.68333333 0.18333333 0.01666667]
the technolog can be use to monitor air craft . (top topic: [8 9 7])
[0.74571429 0.20285714 0.03142857]
the devic could be use in a detect manner that is to say that it could be use to predict behavior pattern of say crimin offend , student , to perform research perhap by track the physic live of athlet or top achiev businessman or for compani to isol expect behavior of their employe and how they anticip employe to move during a work day to establish fair and true standard (top topic: [8 6 9])
[0.61071429 0.14642857 0.11071429]
an applic could be creat for pet owner who have pet who have ran away from home . the applic could be download right onto a person phone . the hand size devic could be put into a collar of a pet , or a chip like devic under the skin . the system would be abl to pin point a pet locat to the direct coordin . (top topic: [8 3 6])
[0.7625 0.1375 0.0125]
the technolog can assist with traffic manag by analyz and predict pattern . (to

In [26]:
from nltk.wsd import lesk
from nltk.corpus import wordnet as wn

In [33]:
sent = nltk.word_tokenize(content_array[0])
print(sent)
a = lesk(sent, 'monitor', 'n')
print(a, a.definition())


['the', 'technolog', 'can', 'be', 'use', 'to', 'monitor', 'air', 'craft', '.']
Synset('monitor.n.05') electronic equipment that is used to check the quality or content of electronic transmissions


In [4]:
for ss in wn.synsets('bank'):
...     print(ss, ss.definition())

Synset('bank.n.01') sloping land (especially the slope beside a body of water)
Synset('depository_financial_institution.n.01') a financial institution that accepts deposits and channels the money into lending activities
Synset('bank.n.03') a long ridge or pile
Synset('bank.n.04') an arrangement of similar objects in a row or in tiers
Synset('bank.n.05') a supply or stock held in reserve for future use (especially in emergencies)
Synset('bank.n.06') the funds held by a gambling house or the dealer in some gambling games
Synset('bank.n.07') a slope in the turn of a road or track; the outside is higher than the inside in order to reduce the effects of centrifugal force
Synset('savings_bank.n.02') a container (usually with a slot in the top) for keeping money at home
Synset('bank.n.09') a building in which the business of banking transacted
Synset('bank.n.10') a flight maneuver; aircraft tips laterally about its longitudinal axis (especially in turning)
Synset('bank.v.01') tip laterally
Sy

In [36]:
from pywsd import disambiguate
from pywsd.lesk import simple_lesk

In [38]:
sent = content_array[0]
print(sent)
a = simple_lesk(sent, 'monitor', 'n')
print(a, a.definition())

the technolog can be use to monitor air craft .
Synset('monitor.n.05') electronic equipment that is used to check the quality or content of electronic transmissions
